# Zip

In [0]:
import os
import requests, zipfile, io
from IPython.display import display

def runzip(url, dpath=".", names=None, peek=False, force=False):
  """
  Retrieve file from url and unzip names to dpath
  peek is True: show members before extracting
  force is True: overwrite existing files/directories
  """
  
  # Retrieve url
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
    
  # Members to extract
  if not names:
    members = z.namelist()
  else:
    members = [p for p in z.namelist() if os.path.basename(p) in names]
  if peek: # show members
    display("Extract", members)
    dparh = input("To {} (or specify other directory):".format(dpath)) or dpath
  
  # Extract if not exist or in force mode
  paths = []
  for m in members:
    p = os.path.join(dpath, m)
    if not os.path.exists(p) or force:
      z.extract(m, dpath)
    else:
      print("{} already exists".format(p))
    paths.append(os.path.abspath(p))
    
  return paths # paths to extracted/existing

# url = "https://prod-edxapp.edx-cdn.org/assets/courseware/v1/73b15fa99e20c9f31078477dff6d80ed/asset-v1:UCSanDiegoX+DSE220x+1T2019+type@asset+block/winery-univariate.zip"
# runzip(url, peek=True)